In [15]:
def compute_recall(groundtruth, detection, score, overlap_ratio):
    #print("==================")
    tp_idx = []   
    for idx_gt, gt in enumerate(groundtruth):
        #print("gt={}".format(gt))
        area = (gt[3] - gt[1]) * (gt[2] - gt[0])
        for idx, dt in enumerate(detection):
            #print("dt = {}".format(dt))
            x2 = min(max(dt[0], dt[2]), max(gt[0], gt[2]))
            y2 = min(max(dt[1], dt[3]), max(gt[1], gt[3]))
            x1 = max(min(dt[0], dt[2]), min(gt[0], dt[2]))
            y1 = max(min(dt[1], dt[3]), min(gt[1], gt[3]))
            #print("y_diff = {}".format(y2-y1))
            #print("x_diff = {}".format(x2-x1))
            if y2 > y1 and x2 > x1:
                overlap = float((y2-y1)*(x2-x1)) / float(area)
                #print("overlap = {}".format(overlap))
                #print("overlap_ratio = {}".format(overlap_ratio))
                #print("dt[4] = {}".format(dt[4]))
                #print("score = {}".format(score))
                if (overlap > overlap_ratio and dt[4] > score):
                    tp_idx.append(idx)
                    break
        
    recall = float(len(tp_idx)) / float(len(groundtruth))
    return recall, tp_idx

def compute_pos(detection, score):
    tp_np_idx = []
    for idx, dt in enumerate(detection):
        if (dt[4] > score):
            tp_np_idx.append(idx)
    return tp_np_idx
            

#----------- get classification result -------------#
file=open("overlap_res_list_github_with_rotation_new_70000.txt")
#file=open("res_list_github_with_rotation_new_70000.txt")
context = file.read()
lines = context.splitlines()

class_dict = {}
for idx, line in enumerate(lines):
    result = line.split("/")
    name = result[3]
    score = float(result[4].split(" ")[-1])
    box = result[4].split(" ")[0].split(".")[0].split("_")
    #print(name)
    #print(score)
    #print(box)
    y = int(box[0])
    x = int(box[1])
    bbox = [x, y, x+512, y+512, score]
    #print("name = {}".format(name))
    #print("--={}".format(class_dict.has_key(name)))
    
    if name in class_dict:
        class_dict[name].append(bbox)
    else:
        cur_list = []
        cur_list.append(bbox)
        class_dict[name] = cur_list
#print(class_dict["2017-06-15_16_09_39"])

#-------------get gt info-------------#
gt_dict = {}
file=open("test_roi_info.txt")
context = file.read()
lines = context.splitlines()
for idx, line in enumerate(lines):
    result = line.split("/")[-1].split(" ")
    name = result[0] + " " + result[1].split(".")[0]
    bbox = [int(result[2].split(".")[0]), int(result[3].split(".")[0]), 
            int(result[4].split(".")[0]), int(result[5].split(".")[0])]
    #print(result[2])
    #print(name)
    #print(bbox)
    
    if name in gt_dict:
        gt_dict[name].append(bbox)
    else:
        cur_list = []
        cur_list.append(bbox)
        gt_dict[name] = cur_list
#print(gt_dict["2017-06-15_16_09_39"])    
    
#-------------set thresholds-------------#
def get_thresh(step):
    num = 0
    thresh = []
    while num < 1:
        thresh.append(num)
        num += step
    return thresh

thresh_score = get_thresh(0.05)
thresh_overlap_ratio = get_thresh(0.05)
thresh_num_ratio = get_thresh(0.05)
thresh_score = [0.5]
thresh_overlap_ratio = [0]
thresh_num_ratio = [0.01]

#-------------compute result-------------#
total_neg_slice = len(class_dict) - len(gt_dict)
total_pos_slice = len(gt_dict)
print("total_neg_slice = {}".format(total_neg_slice))
print("total_neg_slice = {}".format(total_neg_slice))

dict_res = []
res_bbox_recall = []
for idx_score, score in enumerate(thresh_score):
    for idx_overlap_ratio, overlap_ratio in enumerate(thresh_overlap_ratio):
        for idx_num_ratio, num_ratio in enumerate(thresh_num_ratio):
        
            bbox_recall = []
            pos_slice_idx = []
            num_dt_neg_slice = 0
            num_tp_pos_slice = 0
            num_tp_neg_slice = 0
            for key in class_dict.keys():
                
                #for bbox_idx, bbox in enumerate class_dict["key"]:
                #if key != "2018-04-21_13_29_55":
                #    continue;
                
                if key in gt_dict:
                    recall, tp_list = compute_recall(gt_dict[key], class_dict[key], score, overlap_ratio)                    
                else:
                    recall = 0
                    tp_list = []
                bbox_recall.append([key, recall])
                
                print("--------------------")
                print("key = ", key)
                print("recall = {}".format(recall))
                print("pos = {}".format(tp_list))                
                tp_np_list = compute_pos(class_dict[key], score)
                positive_ratio = float(len(tp_np_list)) / float(len(class_dict[key]))
                print("pos = {}".format(len(tp_np_list)))
                print("total = {}".format(len(class_dict[key])))
                print("positive_ratio = {}".format(positive_ratio))
                if positive_ratio - num_ratio < 0:
                    num_neg_slice += 1 
                    if key not in gt_dict:
                        num_tp_neg_slice += 1
                else:
                    pos_slice_idx.append(key)
                    if key in gt_dict:
                        num_tp_pos_slice += 1
            
            print("num_neg_slice = {}".format(num_neg_slice))
            print("num_tp_neg_slice = {}".format(num_tp_neg_slice))
            
            pos_slice_recall_ratio = float(num_tp_pos_slice) / float(total_pos_slice)
            neg_slice_exclude_ratio = float(num_tp_neg_slice) / float(total_neg_slice)
            res = [score, overlap_ratio, num_ratio, pos_slice_recall_ratio, neg_slice_exclude_ratio, pos_slice_idx]
            dict_res.append(res)
            res_bbox_recall.append(bbox_recall)
            print(dict_res)
            print("--------------")
            print(res_bbox_recall)
            

total_neg_slice = 10
total_neg_slice = 10
--------------------
key =  2017-06-30 10_20_21
recall = 0
pos = []
pos = 245
total = 80655
positive_ratio = 0.003037629409212076
--------------------
key =  2018-04-20 14_41_17
recall = 0
pos = []
pos = 148
total = 85769
positive_ratio = 0.0017255651808928635
--------------------
key =  2017-06-15 15_56_02
recall = 1.0
pos = [25975, 13542, 11031, 29259, 30002, 11895, 17788, 26513, 3248, 4046, 68723, 28111, 65275, 323, 4046, 25318, 8726, 69187, 26475, 42657]
pos = 1412
total = 80655
positive_ratio = 0.01750666418696919
--------------------
key =  20010006_2018-01-09 20_11_10
recall = 1.0
pos = [11457, 59992, 121455, 56667]
pos = 13436
total = 186126
positive_ratio = 0.07218765782319504
--------------------
key =  20010003_2018-01-09 19_12_32
recall = 1.0
pos = [26043]
pos = 6149
total = 214266
positive_ratio = 0.02869797354689965
--------------------
key =  2017-06-15 16_09_39
recall = 0.8461538461538461
pos = [13817, 41254, 965, 1734, 1734, 57

In [ ]:
def compute_recall(groundtruth, detection, score, overlap_ratio):
    #print("==================")
    tp_idx = []   
    for idx_gt, gt in enumerate(groundtruth):
        #print("gt={}".format(gt))
        area = (gt[3] - gt[1]) * (gt[2] - gt[0])
        for idx, dt in enumerate(detection):
            #print("dt = {}".format(dt))
            x2 = min(max(dt[0], dt[2]), max(gt[0], gt[2]))
            y2 = min(max(dt[1], dt[3]), max(gt[1], gt[3]))
            x1 = max(min(dt[0], dt[2]), min(gt[0], dt[2]))
            y1 = max(min(dt[1], dt[3]), min(gt[1], gt[3]))
            #print("y_diff = {}".format(y2-y1))
            #print("x_diff = {}".format(x2-x1))
            if y2 > y1 and x2 > x1:
                overlap = float((y2-y1)*(x2-x1)) / float(area)
                #print("overlap = {}".format(overlap))
                #print("overlap_ratio = {}".format(overlap_ratio))
                #print("dt[4] = {}".format(dt[4]))
                #print("score = {}".format(score))
                if (overlap > overlap_ratio and dt[4] > score):
                    tp_idx.append(idx)
                    break
        
    recall = float(len(tp_idx)) / float(len(groundtruth))
    return recall, tp_idx

def compute_pos(detection, score):
    tp_np_idx = []
    for idx, dt in enumerate(detection):
        if (dt[4] > score):
            tp_np_idx.append(idx)
    return tp_np_idx
            

#----------- get classification result -------------#
file=open("overlap_res_list_github_with_rotation_new_70000.txt")
#file=open("res_list_github_with_rotation_new_70000.txt")
context = file.read()
lines = context.splitlines()

class_dict = {}
for idx, line in enumerate(lines):
    result = line.split("/")
    name = result[3]
    score = float(result[4].split(" ")[-1])
    box = result[4].split(" ")[0].split(".")[0].split("_")
    #print(name)
    #print(score)
    #print(box)
    y = int(box[0])
    x = int(box[1])
    bbox = [x, y, x+512, y+512, score]
    #print("name = {}".format(name))
    #print("--={}".format(class_dict.has_key(name)))
    
    if name in class_dict:
        class_dict[name].append(bbox)
    else:
        cur_list = []
        cur_list.append(bbox)
        class_dict[name] = cur_list
#print(class_dict["2017-06-15_16_09_39"])

#-------------get gt info-------------#
gt_dict = {}
file=open("test_roi_info.txt")
context = file.read()
lines = context.splitlines()
for idx, line in enumerate(lines):
    result = line.split("/")[-1].split(" ")
    name = result[0] + " " + result[1].split(".")[0]
    bbox = [int(result[2].split(".")[0]), int(result[3].split(".")[0]), 
            int(result[4].split(".")[0]), int(result[5].split(".")[0])]
    #print(result[2])
    #print(name)
    #print(bbox)
    
    if name in gt_dict:
        gt_dict[name].append(bbox)
    else:
        cur_list = []
        cur_list.append(bbox)
        gt_dict[name] = cur_list
#print(gt_dict["2017-06-15_16_09_39"])    
    
#-------------set thresholds-------------#
def get_thresh(min_value, max_value, step):
    num = min_value
    thresh = []
    while num - max_value < 0:
        thresh.append(num)
        num += step
    return thresh

thresh_score = get_thresh(0.3, 0.7, 0.05)
thresh_overlap_ratio = get_thresh(0, 0.5, 0.05)
thresh_num_ratio = get_thresh(0, 0.05, 0.002)
#thresh_score = [0.5]
#thresh_overlap_ratio = [0]
#thresh_num_ratio = [0.01]

#-------------compute result-------------#
total_neg_slice = len(class_dict) - len(gt_dict)
total_pos_slice = len(gt_dict)
#print("total_neg_slice = {}".format(total_neg_slice))
#print("total_neg_slice = {}".format(total_neg_slice))
dict_res = []
res_bbox_recall = []
for idx_score, score in enumerate(thresh_score):
    for idx_overlap_ratio, overlap_ratio in enumerate(thresh_overlap_ratio):
        for idx_num_ratio, num_ratio in enumerate(thresh_num_ratio):
        
            bbox_recall = []
            pos_slice_idx = []
            num_dt_neg_slice = 0
            num_tp_pos_slice = 0
            num_tp_neg_slice = 0
            for key in class_dict.keys():
                
                #for bbox_idx, bbox in enumerate class_dict["key"]:
                #if key != "2018-04-21_13_29_55":
                #    continue;
                
                if key in gt_dict:
                    recall, tp_list = compute_recall(gt_dict[key], class_dict[key], score, overlap_ratio)                    
                else:
                    recall = 0
                    tp_list = []
                bbox_recall.append([key, recall])
                
                #print("--------------------")
                #print("key = ", key)
                #print("recall = {}".format(recall))
                #print("pos = {}".format(tp_list))                
                tp_np_list = compute_pos(class_dict[key], score)
                positive_ratio = float(len(tp_np_list)) / float(len(class_dict[key]))
                #print("pos = {}".format(len(tp_np_list)))
                #print("total = {}".format(len(class_dict[key])))
                #print("positive_ratio = {}".format(positive_ratio))
                if positive_ratio - num_ratio < 0:
                    num_neg_slice += 1 
                    if key not in gt_dict:
                        num_tp_neg_slice += 1
                else:
                    pos_slice_idx.append(key)
                    if key in gt_dict:
                        num_tp_pos_slice += 1
            
            #print("num_neg_slice = {}".format(num_neg_slice))
            #print("num_tp_neg_slice = {}".format(num_tp_neg_slice))
            
            pos_slice_recall_ratio = float(num_tp_pos_slice) / float(total_pos_slice)
            neg_slice_exclude_ratio = float(num_tp_neg_slice) / float(total_neg_slice)
            res = [score, overlap_ratio, num_ratio, pos_slice_recall_ratio, neg_slice_exclude_ratio, pos_slice_idx]
            dict_res.append(res)
            res_bbox_recall.append(bbox_recall)
            print("-----------------")
            print(res)
print(dict_res)
print(bbox_recall)

-----------------
[0.3, 0, 0, 1.0, 0.0, ['2017-06-30 10_20_21', '2018-04-20 14_41_17', '2017-06-15 15_56_02', '20010006_2018-01-09 20_11_10', '20010003_2018-01-09 19_12_32', '2017-06-15 16_09_39', '2018-04-19 15_27_41', '2018-04-21 11_45_52', '2018-04-21 14_32_37', '2018-04-21 13_29_55', '2018-04-21 03_49_19', '2018-04-21 03_48_14', '2018-04-21 13_27_42', '2018-04-20 12_26_31', '2018-04-19 13_06_14', '2018-04-19 12_20_27', '2018-04-21 02_23_36', '2018-04-21 02_12_32', '2018-04-19 14_36_52', '2018-04-21 00_57_40']]
-----------------
[0.3, 0, 0.002, 1.0, 0.2, ['2017-06-30 10_20_21', '2018-04-20 14_41_17', '2017-06-15 15_56_02', '20010006_2018-01-09 20_11_10', '20010003_2018-01-09 19_12_32', '2017-06-15 16_09_39', '2018-04-19 15_27_41', '2018-04-21 14_32_37', '2018-04-21 13_29_55', '2018-04-21 03_49_19', '2018-04-21 03_48_14', '2018-04-20 12_26_31', '2018-04-19 13_06_14', '2018-04-19 12_20_27', '2018-04-21 02_23_36', '2018-04-21 02_12_32', '2018-04-19 14_36_52', '2018-04-21 00_57_40']]
--

-----------------
[0.3, 0.05, 0.006, 1.0, 0.8, ['2017-06-15 15_56_02', '20010006_2018-01-09 20_11_10', '20010003_2018-01-09 19_12_32', '2017-06-15 16_09_39', '2018-04-19 15_27_41', '2018-04-21 13_29_55', '2018-04-19 13_06_14', '2018-04-19 12_20_27', '2018-04-21 02_23_36', '2018-04-21 02_12_32', '2018-04-19 14_36_52', '2018-04-21 00_57_40']]
-----------------
[0.3, 0.05, 0.008, 1.0, 0.8, ['2017-06-15 15_56_02', '20010006_2018-01-09 20_11_10', '20010003_2018-01-09 19_12_32', '2017-06-15 16_09_39', '2018-04-19 15_27_41', '2018-04-21 13_29_55', '2018-04-19 13_06_14', '2018-04-19 12_20_27', '2018-04-21 02_23_36', '2018-04-21 02_12_32', '2018-04-19 14_36_52', '2018-04-21 00_57_40']]
-----------------
[0.3, 0.05, 0.01, 1.0, 0.8, ['2017-06-15 15_56_02', '20010006_2018-01-09 20_11_10', '20010003_2018-01-09 19_12_32', '2017-06-15 16_09_39', '2018-04-19 15_27_41', '2018-04-21 13_29_55', '2018-04-19 13_06_14', '2018-04-19 12_20_27', '2018-04-21 02_23_36', '2018-04-21 02_12_32', '2018-04-19 14_36_5

-----------------
[0.3, 0.1, 0.014, 1.0, 0.8, ['2017-06-15 15_56_02', '20010006_2018-01-09 20_11_10', '20010003_2018-01-09 19_12_32', '2017-06-15 16_09_39', '2018-04-19 15_27_41', '2018-04-21 13_29_55', '2018-04-19 13_06_14', '2018-04-19 12_20_27', '2018-04-21 02_23_36', '2018-04-21 02_12_32', '2018-04-19 14_36_52', '2018-04-21 00_57_40']]
-----------------
[0.3, 0.1, 0.016, 0.9, 0.8, ['2017-06-15 15_56_02', '20010006_2018-01-09 20_11_10', '20010003_2018-01-09 19_12_32', '2017-06-15 16_09_39', '2018-04-19 15_27_41', '2018-04-21 13_29_55', '2018-04-19 13_06_14', '2018-04-19 12_20_27', '2018-04-21 02_23_36', '2018-04-21 02_12_32', '2018-04-21 00_57_40']]
-----------------
[0.3, 0.1, 0.018000000000000002, 0.7, 0.8, ['2017-06-15 15_56_02', '20010006_2018-01-09 20_11_10', '20010003_2018-01-09 19_12_32', '2017-06-15 16_09_39', '2018-04-19 15_27_41', '2018-04-21 13_29_55', '2018-04-19 12_20_27', '2018-04-21 02_23_36', '2018-04-21 02_12_32']]
-----------------
[0.3, 0.1, 0.020000000000000004, 

-----------------
[0.3, 0.15000000000000002, 0.022000000000000006, 0.6, 0.8, ['2017-06-15 15_56_02', '20010006_2018-01-09 20_11_10', '20010003_2018-01-09 19_12_32', '2018-04-19 15_27_41', '2018-04-21 13_29_55', '2018-04-19 12_20_27', '2018-04-21 02_23_36', '2018-04-21 02_12_32']]
-----------------
[0.3, 0.15000000000000002, 0.024000000000000007, 0.5, 0.8, ['20010006_2018-01-09 20_11_10', '20010003_2018-01-09 19_12_32', '2018-04-19 15_27_41', '2018-04-21 13_29_55', '2018-04-19 12_20_27', '2018-04-21 02_23_36', '2018-04-21 02_12_32']]
-----------------
[0.3, 0.15000000000000002, 0.02600000000000001, 0.5, 0.8, ['20010006_2018-01-09 20_11_10', '20010003_2018-01-09 19_12_32', '2018-04-19 15_27_41', '2018-04-21 13_29_55', '2018-04-19 12_20_27', '2018-04-21 02_23_36', '2018-04-21 02_12_32']]
-----------------
[0.3, 0.15000000000000002, 0.02800000000000001, 0.5, 0.8, ['20010006_2018-01-09 20_11_10', '20010003_2018-01-09 19_12_32', '2018-04-19 15_27_41', '2018-04-21 13_29_55', '2018-04-19 12_20

-----------------
[0.3, 0.2, 0.030000000000000013, 0.5, 0.8, ['20010006_2018-01-09 20_11_10', '20010003_2018-01-09 19_12_32', '2018-04-19 15_27_41', '2018-04-21 13_29_55', '2018-04-19 12_20_27', '2018-04-21 02_23_36', '2018-04-21 02_12_32']]
-----------------
[0.3, 0.2, 0.032000000000000015, 0.5, 0.8, ['20010006_2018-01-09 20_11_10', '20010003_2018-01-09 19_12_32', '2018-04-19 15_27_41', '2018-04-21 13_29_55', '2018-04-19 12_20_27', '2018-04-21 02_23_36', '2018-04-21 02_12_32']]
-----------------
[0.3, 0.2, 0.034000000000000016, 0.5, 0.9, ['20010006_2018-01-09 20_11_10', '20010003_2018-01-09 19_12_32', '2018-04-21 13_29_55', '2018-04-19 12_20_27', '2018-04-21 02_23_36', '2018-04-21 02_12_32']]
-----------------
[0.3, 0.2, 0.03600000000000002, 0.5, 0.9, ['20010006_2018-01-09 20_11_10', '20010003_2018-01-09 19_12_32', '2018-04-21 13_29_55', '2018-04-19 12_20_27', '2018-04-21 02_23_36', '2018-04-21 02_12_32']]
-----------------
[0.3, 0.2, 0.03800000000000002, 0.5, 0.9, ['20010006_2018-01-

...

-----------------
[0.5499999999999999, 0.1, 0.002, 1.0, 0.5, ['2017-06-30 10_20_21', '2017-06-15 15_56_02', '20010006_2018-01-09 20_11_10', '20010003_2018-01-09 19_12_32', '2017-06-15 16_09_39', '2018-04-19 15_27_41', '2018-04-21 13_29_55', '2018-04-21 03_49_19', '2018-04-20 12_26_31', '2018-04-19 13_06_14', '2018-04-19 12_20_27', '2018-04-21 02_23_36', '2018-04-21 02_12_32', '2018-04-19 14_36_52', '2018-04-21 00_57_40']]
-----------------
[0.5499999999999999, 0.1, 0.004, 1.0, 0.8, ['2017-06-15 15_56_02', '20010006_2018-01-09 20_11_10', '20010003_2018-01-09 19_12_32', '2017-06-15 16_09_39', '2018-04-19 15_27_41', '2018-04-21 13_29_55', '2018-04-19 13_06_14', '2018-04-19 12_20_27', '2018-04-21 02_23_36', '2018-04-21 02_12_32', '2018-04-19 14_36_52', '2018-04-21 00_57_40']]
-----------------
[0.5499999999999999, 0.1, 0.006, 1.0, 0.8, ['2017-06-15 15_56_02', '20010006_2018-01-09 20_11_10', '20010003_2018-01-09 19_12_32', '2017-06-15 16_09_39', '2018-04-19 15_27_41', '2018-04-21 13_29_55',

-----------------
[0.5499999999999999, 0.15000000000000002, 0.016, 0.6, 0.8, ['2017-06-15 15_56_02', '20010006_2018-01-09 20_11_10', '20010003_2018-01-09 19_12_32', '2018-04-19 15_27_41', '2018-04-21 13_29_55', '2018-04-19 12_20_27', '2018-04-21 02_23_36', '2018-04-21 02_12_32']]
-----------------
[0.5499999999999999, 0.15000000000000002, 0.018000000000000002, 0.5, 0.8, ['20010006_2018-01-09 20_11_10', '20010003_2018-01-09 19_12_32', '2018-04-19 15_27_41', '2018-04-21 13_29_55', '2018-04-19 12_20_27', '2018-04-21 02_23_36', '2018-04-21 02_12_32']]
-----------------
[0.5499999999999999, 0.15000000000000002, 0.020000000000000004, 0.5, 0.9, ['20010006_2018-01-09 20_11_10', '20010003_2018-01-09 19_12_32', '2018-04-21 13_29_55', '2018-04-19 12_20_27', '2018-04-21 02_23_36', '2018-04-21 02_12_32']]
-----------------
[0.5499999999999999, 0.15000000000000002, 0.022000000000000006, 0.5, 0.9, ['20010006_2018-01-09 20_11_10', '20010003_2018-01-09 19_12_32', '2018-04-21 13_29_55', '2018-04-19 12_2

-----------------
[0.5499999999999999, 0.2, 0.04200000000000002, 0.2, 1.0, ['20010006_2018-01-09 20_11_10', '2018-04-21 02_23_36']]
-----------------
[0.5499999999999999, 0.2, 0.044000000000000025, 0.2, 1.0, ['20010006_2018-01-09 20_11_10', '2018-04-21 02_23_36']]
-----------------
[0.5499999999999999, 0.2, 0.04600000000000003, 0.2, 1.0, ['20010006_2018-01-09 20_11_10', '2018-04-21 02_23_36']]
-----------------
[0.5499999999999999, 0.2, 0.04800000000000003, 0.2, 1.0, ['20010006_2018-01-09 20_11_10', '2018-04-21 02_23_36']]
-----------------
[0.5499999999999999, 0.25, 0, 1.0, 0.0, ['2017-06-30 10_20_21', '2018-04-20 14_41_17', '2017-06-15 15_56_02', '20010006_2018-01-09 20_11_10', '20010003_2018-01-09 19_12_32', '2017-06-15 16_09_39', '2018-04-19 15_27_41', '2018-04-21 11_45_52', '2018-04-21 14_32_37', '2018-04-21 13_29_55', '2018-04-21 03_49_19', '2018-04-21 03_48_14', '2018-04-21 13_27_42', '2018-04-20 12_26_31', '2018-04-19 13_06_14', '2018-04-19 12_20_27', '2018-04-21 02_23_36', '20

-----------------
[0.5499999999999999, 0.3, 0.01, 0.8, 0.8, ['2017-06-15 15_56_02', '20010006_2018-01-09 20_11_10', '20010003_2018-01-09 19_12_32', '2017-06-15 16_09_39', '2018-04-19 15_27_41', '2018-04-21 13_29_55', '2018-04-19 12_20_27', '2018-04-21 02_23_36', '2018-04-21 02_12_32', '2018-04-19 14_36_52']]
-----------------
[0.5499999999999999, 0.3, 0.012, 0.7, 0.8, ['2017-06-15 15_56_02', '20010006_2018-01-09 20_11_10', '20010003_2018-01-09 19_12_32', '2017-06-15 16_09_39', '2018-04-19 15_27_41', '2018-04-21 13_29_55', '2018-04-19 12_20_27', '2018-04-21 02_23_36', '2018-04-21 02_12_32']]
-----------------
[0.5499999999999999, 0.3, 0.014, 0.6, 0.8, ['2017-06-15 15_56_02', '20010006_2018-01-09 20_11_10', '20010003_2018-01-09 19_12_32', '2018-04-19 15_27_41', '2018-04-21 13_29_55', '2018-04-19 12_20_27', '2018-04-21 02_23_36', '2018-04-21 02_12_32']]
-----------------
[0.5499999999999999, 0.3, 0.016, 0.6, 0.8, ['2017-06-15 15_56_02', '20010006_2018-01-09 20_11_10', '20010003_2018-01-09

-----------------
[0.5499999999999999, 0.35, 0.032000000000000015, 0.3, 1.0, ['20010006_2018-01-09 20_11_10', '2018-04-19 12_20_27', '2018-04-21 02_23_36']]
-----------------
[0.5499999999999999, 0.35, 0.034000000000000016, 0.2, 1.0, ['20010006_2018-01-09 20_11_10', '2018-04-21 02_23_36']]
-----------------
[0.5499999999999999, 0.35, 0.03600000000000002, 0.2, 1.0, ['20010006_2018-01-09 20_11_10', '2018-04-21 02_23_36']]
-----------------
[0.5499999999999999, 0.35, 0.03800000000000002, 0.2, 1.0, ['20010006_2018-01-09 20_11_10', '2018-04-21 02_23_36']]
-----------------
[0.5499999999999999, 0.35, 0.04000000000000002, 0.2, 1.0, ['20010006_2018-01-09 20_11_10', '2018-04-21 02_23_36']]
-----------------
[0.5499999999999999, 0.35, 0.04200000000000002, 0.2, 1.0, ['20010006_2018-01-09 20_11_10', '2018-04-21 02_23_36']]
-----------------
[0.5499999999999999, 0.35, 0.044000000000000025, 0.2, 1.0, ['20010006_2018-01-09 20_11_10', '2018-04-21 02_23_36']]
-----------------
[0.5499999999999999, 0.35

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

dict_index_param={}
idx = []
neg_slice_exclude_rate = []

num = 0
for i in range(len(dict_res)):
    if (int(dict_res[i,3]) == 1):
        print(dict_res[i])
        dict_index_param[num].append([dict_res[i, 0:3]])
        idx.append(num)
        neg_slice_exclude_rate.append([dict_res[i, 4]])
        
max_index = neg_slice_exclude_rate.index(max(neg_slice_exclude_rate))
best_param = dict_index_param[max_index]

axis_x = np.array(idx)
axis_y = np.array(neg_slice_exclude_rate)
plt.plot(axis_x, axis_y)
plt.show()